Imports

In [4]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
torch.cuda.is_available()

True

Download datasets

In [9]:
# add download of ModelNet40 in data/ModelNet40
print('Downloading ModelNet40...')
!wget modelnet.cs.princeton.edu/ModelNet40.zip -P data
print('Extracting ModelNet40...')
!unzip -q data/ModelNet40.zip -d data
!rm data/ModelNet40.zip
print('ModelNet40 done')

# add download of 7Scenes in data/tsdf
print('Downloading 7Scenes...')
!wget download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip -P data
print('Extracting 7Scenes...')
!unzip -q data/tsdf.zip -d data
!rm data/tsdf.zip
print('7Scenes done')

--2021-12-12 22:01:08--  http://modelnet.cs.princeton.edu/ModelNet40.zip
Resolving modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)... 128.112.136.61
Connecting to modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)|128.112.136.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2039180837 (1.9G) [application/zip]
Saving to: ‘data/ModelNet40.zip’

ModelNet40.zip      100%[===================>]   1.90G  3.24MB/s    in 45m 0s  

2021-12-12 22:46:08 (738 KB/s) - ‘data/ModelNet40.zip’ saved [2039180837/2039180837]

Extracting ModelNet40...
ModelNet40 done
--2021-12-12 22:47:04--  http://download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip
Resolving download.microsoft.com (download.microsoft.com)... 2a02:26f0:3500:595::e59, 2a02:26f0:3500:583::e59, 23.213.164.137
Connecting to download.microsoft.com (download.microsoft.com)|2a02:26f0:3500:595::e59|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Loc

Run simple configuration

In [7]:
# run simple configuration

Visualize results

In [8]:
# add visualization